In [ ]:
# !pip install import_ipynb --quiet
# !pip install anvil-uplink --quiet
# !pip install yfinance --quiet
# !pip install pandas_ta --quiet
# !pip install ipynb --quiet
# !pip install rpyc --quiet
# !pip install stable-baselines3 --quiet
# !pip install aspectlib

In [ ]:
# !git clone https://github.com/gmshroff/algostrats.git

In [ ]:
# %cd algostrats

Uncomment below if on Colab and using datasets from Kaggle

In [ ]:
# from google.colab import files
# uploaded=files.upload()

In [ ]:
# !mkdir /root/.kaggle
# !mv ./kaggle.json /root/.kaggle/.
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# %mkdir data
# %cd data
# !kaggle datasets download -d gmshroff/marketdatafivemin
# %cd ../algostrats

In [1]:
colab=False
script=False
if not colab: 
    DATAPATH='../algodata/'
elif colab: DATAPATH='../data/'

Need to import algorithms from stable-baselines3

In [2]:
from stable_baselines3 import PPO,A2C,DQN
from stable_baselines3.common.vec_env import StackedObservations
from stable_baselines3.common.monitor import Monitor as Mon

In [3]:
import warnings
warnings.simplefilter("ignore")

In [4]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle
from threading import Thread
import threading
from IPython import display
import time,getopt,sys,os

In [5]:
from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb


In [6]:
from feed_env import Episode
import aspectlib

importing Jupyter notebook from feed_env.ipynb


### Trading strategies as agents

RL(++)StratAgents imported from ruleagents<br>

In [7]:
from rlagents import RLStratAgentDyn, COLS

importing Jupyter notebook from rlagents.ipynb
importing Jupyter notebook from aiagentbase.ipynb


In [8]:
len(COLS)

8853

### Strategy Development: Training RLStratAgent using BackTestWorld

In [9]:
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate
from add_features import add_features

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from validation.ipynb


<b>Configuration for training RL model<b>

In [10]:
algorithm=PPO
synthetic=False #use synthetic data
simple='sinewave' #False,True or 'sinewave'
nd,nw=4,5 #for BackFeed

In [11]:
if script:
    try:
        opts,args = getopt.getopt(sys.argv[1:],"hl:f:d:m:s:w:t:p:u:",["load=","feed=","datafile=","modelname=","synthetic","weeks","training_steps","deploy","use_alt_data"])
    except getopt.GetoptError:
        print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy>')
        sys.exit(2)
    load,feed,modelname=False,'back',''
    training_steps=50000 # if less then n_steps then n_steps is used
    deploy=True
    date=datetime.today().strftime('%d-%b-%Y')
    use_alt_data=False
    for opt, arg in opts:
        if opt == "-h":
            print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy> -u <use_alt_data>')
            sys.exit()
        elif opt in ("-l", "--load"):
            load = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-f", "--feed"):
            feed = (lambda x: 'data' if x=='data' else 'back')(arg)
        elif opt in ("-d", "--datafile"):
            datafile = arg.split('/')[-1]
        elif opt in ("-m", "--modelname"):
            modelname = arg
        elif opt in ("-s", "--synthetic"):
            synthetic = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-w", "--weeks"):
            nw = int(arg)
        elif opt in ("-t", "--training_steps"):
            training_steps=int(arg)
        elif opt in ("-p", "--deploy"):
            deploy = (lambda x: True if x=='True' else False)(arg)
        elif opt in ("-u", "--use_alt_data"):
            use_alt_data = (lambda x: True if x=='True' else False)(arg)
    if len(opts)==0: 
        print('rlagents_train.py -l <load:True/False> -f <scan:back/data> -d <datafile> -m <modelname> -s <synthetic> -w <weeks> -t <training_steps> -p <deploy> -u <use_alt_data>')
        sys.exit()
    print(f"load:{load},feed:{feed},datafile:{datafile},modelname:{modelname},synthetic:{synthetic},weeks:{nw},training_steps:{training_steps},deploy:{deploy},use_alt_data:{use_alt_data}")
    loadfeed=load
    if feed=='data': datafeed=True
    else: datafeed=False

In [12]:
if not script:
    loadfeed=False
    datafeed=True
    datafile='realdata/alldata.csv'
    modelname='RLC0_all.pth'
    # modelname='SINE1.pth' # replace with modelname if model to be saved to saved_models
    # modelname='RLG0.pth'
    date=datetime.today().strftime('%d-%b-%Y')
    training_steps=1000000 # if less then n_steps then n_steps is used
    deploy=True
    use_alt_data=False

In [13]:
n_steps=2048 # reduce for debugging only else 2048

In [14]:
def stringify(x):
    return pd.to_datetime(x['Datetime']).strftime('%d-%b-%Y')

In [15]:
ADD_FEATURES = True
RETAIN_OLD_FEATURES = True

In [16]:
import pickle
if not loadfeed and not datafeed:
    data=pd.read_csv('./capvolfiltered.csv')
    tickers=list(data.iloc[0:50]['ticker'].values)
    print('Creating feed')
    feed=BackFeed(tickers=tickers,nd=nd,nw=nw,interval='5m',synthetic=synthetic,simple=simple)
    print('Processing feed')
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    if not synthetic: add_global_indices_feed(feed)
    if not colab: 
        with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)
    elif colab: 
        with open('/tmp/btfeed.pickle','wb') as f: pickle.dump(feed,f)
elif loadfeed and not datafeed:
    if not colab: 
        with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)
    elif colab: 
        with open('/tmp/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [17]:
if not loadfeed and datafeed:
    DATAFILE=DATAPATH+datafile
    print('Reading datafile')
    df=pd.read_csv(DATAFILE)
    if 'Date' not in df.columns: 
        print('Adding Date')
        df['Date']=df.apply(stringify,axis=1)
    print('Creating feed')
    feed=DataFeed(tickers=list(df.ticker.unique()[0:10]),dfgiven=True,df=df)
    print('Processing feed')
    
    add_addl_features_feed(feed,tickers=feed.tickers)
    add_sym_feature_feed(feed,tickers=feed.tickers)
    # add_global_indices_feed(feed)
    
    if add_features:
        for ticker in feed.data:
            df = feed.data[ticker]
            df, pre_discrete_cols, discrete_cols = add_features(df)
            feed.data[ticker] = df
            
        for t in feed.ndata:
            for d in feed.ndata[t]:
                df = feed.ndata[t][d]
                df, _, _ = add_features(df)
                feed.ndata[t][d] = df

    if not colab: 
        with open(f'../algodata/realdata/datafeed_alldata_{RETAIN_OLD_FEATURES}_{ADD_FEATURES}.pkl','wb') as f: 
            pickle.dump(feed,f)
    elif colab: 
        with open('/tmp/btdatafeed.pickle','wb') as f: pickle.dump(feed,f)
        
        
elif loadfeed and datafeed:
    if not colab: 
        with open(f'../algodata/realdata/datafeed_alldata_{RETAIN_OLD_FEATURES}_{ADD_FEATURES}.pkl','rb') as f: feed=pickle.load(f)
    elif colab:
        with open('/tmp/btdatafeed.pickle','rb') as f: feed=pickle.load(f)

Reading datafile
Creating feed
Processing feed


KeyboardInterrupt: 

In [18]:
# pd.options.mode.use_inf_as_na = True
for t in feed.ndata:
    for d in feed.ndata[t]:
        if feed.ndata[t][d].isnull().values.any(): 
            feed.ndata[t][d]=feed.ndata[t][d].fillna(1)
            # print(t,d)
        if feed.ndata[t][d].isin([-np.inf,np.inf]).values.any():
            feed.ndata[t][d]=feed.ndata[t][d].replace([np.inf, -np.inf],1)
            # print(t,d)

In [19]:
# with open('additional_utils/cols.pkl', 'wb') as f:
#     disc_wo_original = [col for col in discrete_cols if '_1_' not in col]
#     print(len(discrete_cols), len(disc_wo_original))
#     dic = {'pre_discrete_cols': pre_discrete_cols, 'discrete_cols': discrete_cols,
#            'discrete_wo_origninal_cols': disc_wo_original}
#     pickle.dump(dic, f)

In [20]:
# from math import isnan
# for d in feed.gdata:
#     for g in feed.gdata[d]:
#          for k in g: 
#                 if isnan(g[k]): g[k]=1

In [21]:
def get_alt_data_live():
    aD={'gdata':feed.gdata}
    return aD

In [22]:
agent=RLStratAgentDyn(algorithm,monclass=Mon,soclass=StackedObservations,verbose=1,win=5,
                   metarl=True,myargs=(n_steps,use_alt_data))
agent.use_memory=True #depends on whether RL algorithm uses memory for state computation
agent.debug=False
if use_alt_data: agent.set_alt_data(alt_data_func=get_alt_data_live)

Using cpu device
Wrapping the env in a DummyVecEnv.


In [23]:
if modelname and os.path.exists('./saved_models/'+modelname):
    print('Loading model')
    agent.load_model(filepath='./saved_models/'+modelname)

In [24]:
@aspectlib.Aspect
def my_decorator(*args, **kwargs):
    # print("Got called with args: %s kwargs: %s" % (args, kwargs))
    # result = yield
    # print(" ... and the result is: %s" % (result,))
    state,rew,done,exit_type = yield
    # args[0].policy.logL+=[(state.keys(),rew,done,exit_type)]
    args[0].policy.reward((rew,done,{'exit_type':exit_type}))
    return state,rew,done,exit_type

In [25]:
aspectlib.weave(Episode, my_decorator, methods='env_step')

In [26]:
bt=Backtest(feed,tickers=feed.tickers,add_features=False,target=5,stop=5,txcost=0.001,
            loc_exit=True,scan=True,topk=5,deploy=deploy,save_dfs=False,
            save_func=None)

In [27]:
agent.data_cols=agent.data_cols+['Date']
agent.error=False

In [28]:
def run_btworld():
    global bt,feed,agent
    while agent.training and not agent.error:
        bt.run_all(tickers=feed.tickers,model=agent,verbose=False)

In [29]:
agent.start(training_steps=training_steps)

In [30]:
btworldthread=Thread(target=run_btworld,name='btworld')
btworldthread.start()

In [32]:
def check_bt_training_status():
    threadL=[thread.name for thread in threading.enumerate()]
    # print(threadL)
    if 'monitor' not in threadL and 'btworld' not in threadL:
        print(f'Training Over after {agent.model.num_timesteps} steps')
        return False
    else:
        print(f'Model Training for {agent.model.num_timesteps} steps')
        return True

In [33]:
while check_bt_training_status():
    time.sleep(2)

Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps
Model Training for 0 steps


KeyboardInterrupt: 

In [ ]:
# Save learned model
# modelname=''
if modelname: torch.save(agent.model.policy.state_dict(),'./saved_models/'+modelname)

## Training Curves

In [ ]:
if not script:
    import pandas as pd
    df=pd.read_csv('/tmp/aiagents.monitor.csv',comment='#')

In [ ]:
if not script:
    import plotly.express as px
    px.line(df['r'].rolling(window=100).mean().values).show()

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.86       |
|    ep_rew_mean          | -0.48      |
| time/                   |            |
|    fps                  | 19         |
|    iterations           | 7          |
|    time_elapsed         | 732        |
|    total_timesteps      | 14336      |
| train/                  |            |
|    approx_kl            | 0.01605858 |
|    clip_fraction        | 0.178      |
|    clip_range           | 0.2        |
|    entropy_loss         | -3.21      |
|    explained_variance   | 0.00179    |
|    learning_rate        | 0.0003     |
|    loss                 | 2.12       |
|    n_updates            | 60         |
|    policy_gradient_loss | -0.0129    |
|    value_loss           | 3.13       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.19        |
|    ep_rew_m